In [46]:
# hidden num remain unchange

In [1]:
import torch
old = torch.load('/home/workspace/util/surgery/titanic_demo/runs/May24_08-22-52_1d7a14f7cd78old_network with old input from scratch/oldnetwork_oldinput_from_scratch_bs32.ckpt')

In [2]:
# hidden num remain unchange
# import os
import numpy as np
# from torch import optim
from pretreat import *
%matplotlib inline
np.random.seed(0)
training = get_pretreated_training_data()
old_input_dict = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'cabine_n', 'IsAlone']
new_input_dict = old_input_dict + ['sp', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
loader = get_dataloader(training, new_input_dict, bs=32)
new_net = MLP(input_dim = len(new_input_dict), hidden_dims=[40, 50, 100, 100, 100, 100,]).cuda()

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [3]:
layer = 0
w = "layers.{}.weight".format(layer)
b = "layers.{}.bias".format(layer)
w1 = old['para'][w]
b1 = old['para'][b]

inwmap = {}
for i, f in enumerate(old['input']):
    inwmap[f] = w1[:, i]

inbmap = {} # 和hidden node 相关
for i in range(w1.shape[0]):
    inbmap[i] = b1[i]

In [4]:
new_net.state_dict()

OrderedDict([('layers.0.weight',
              tensor([[ 0.1024,  0.0263,  0.0415,  0.1118,  0.2587,  0.1949, -0.1335, -0.1673,
                       -0.2020, -0.1017,  0.1835, -0.1012, -0.2724],
                      [-0.0717, -0.0484, -0.1261, -0.0118, -0.0865,  0.1870, -0.0985,  0.0696,
                        0.1426,  0.0292, -0.2346,  0.2012, -0.2456],
                      [ 0.1258, -0.1226,  0.0201,  0.2485, -0.0518,  0.0024, -0.1842, -0.0013,
                       -0.1204, -0.1461,  0.0276,  0.0321,  0.1543],
                      [ 0.0268, -0.1646,  0.2257, -0.0313,  0.0343,  0.2245,  0.0910, -0.2052,
                       -0.0441,  0.1158, -0.2747,  0.1335, -0.2618],
                      [-0.0040, -0.2326, -0.2263,  0.0358,  0.1172,  0.1450,  0.1545, -0.0527,
                       -0.1835,  0.2368, -0.2382,  0.1838, -0.0470],
                      [-0.2068,  0.0014, -0.1838, -0.2324,  0.1587,  0.2762, -0.2317, -0.0708,
                        0.1608, -0.2134,  0.1687, -0

In [51]:
# 增加了输入 和中间层, 对于多出来的进行0初始化, 假设所有多出来的node都在原来hidden的下面同一侧
for i, (name, g) in enumerate(new_net.named_parameters()):
    old_g = old['para'][name].data
    if old_g.shape != g.shape:
        if i == 1:
            # new_g = torch.zeros_like(g)
            # new_g = torch.ones_like(g) * (10 **(-40))
            for hidden_n, b in inbmap.items():
                new_g[hidden_n] = b
        elif i == 0:
            # new_g = torch.zeros_like(g) # 最开始的初始化方式是0初始化所有新权重, 发现因为ReLU的关系, 训练后都为0
            # new_g = torch.ones_like(g) * (10 **(-40)) #  然后使用很小的权重初始化, 训练后不为0了, 但是查看了一下权重, 感觉还是有点问题, 很多都是10**-40左右. 
            for j, f in enumerate(new_input_dict):
                if f in inwmap.keys():
                    old_w = inwmap[f]
                    new_g[:old_w.shape[0], j] = old_w
        elif i == 2:
            new_g = g.clone().detach() # 这里是忘记改了
            new_g[:, :old_g.shape[1]] = old_g
        else:
            assert False
        g.data = new_g
    else:
        # print(name, g.shape)
        g.data = old['para'][name].data
    

In [5]:
# 增加了输入 和中间层, 对于多出来的进行0初始化, 假设所有多出来的node都在原来hidden的下面同一侧
for i, (name, g) in enumerate(new_net.named_parameters()):
    old_g = old['para'][name].data
    if old_g.shape != g.shape:
        if i == 1:
            new_g = g.data * (10**(-20))
            for hidden_n, b in inbmap.items():
                new_g[hidden_n] = b
        elif i == 0:
            new_g = g.data * (10**(-20))
            for j, f in enumerate(new_input_dict):
                if f in inwmap.keys():
                    old_w = inwmap[f]
                    new_g[:old_w.shape[0], j] = old_w
        elif i == 2:
            new_g = torch.zeros_like(g)
            new_g[:, :old_g.shape[1]] = old_g
        else:
            assert False
        g.data = new_g
    else:
        # print(name, g.shape)
        g.data = old['para'][name].data
    

In [6]:
# get surgery之后最初始的acc
sum_acc = 0
for x, y in loader:
    x, y = x.cuda(), y.cuda()
    outputs=new_net(x)
    _,id=torch.max(outputs.data,1)
    acc=torch.sum(id==y.data)
    sum_acc += acc.item() 
print(sum_acc)

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


752


In [7]:
lr = 0.001
epoches = 2000 
comment = 'hidden_add_new_net with new input Using Surgery'
model_name = 'hidden_add_newnetwork_newinput_usingSurgery.ckpt'
input_dic = new_input_dict
acc_cur, loss_cur = train(new_net, loader, comment, model_name, input_dic, )

/home/miniconda/envs/tftorch/lib/python3.6/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch:0, loss:13.298031389713287, acc:0.8383838383838383
epoch:500, loss:11.919799119234085, acc:0.8877665544332211
epoch:1000, loss:12.139749348163605, acc:0.8810325476992144
epoch:1500, loss:12.01339527964592, acc:0.8855218855218855


In [10]:
new_net.state_dict()['layers.0.weight'][25:]
# new_net.state_dict()['layers.0.bias']

tensor([[-6.0361e-41, -4.0102e-16,  1.7307e-41, -4.2621e-40, -5.0089e-22,
          4.8372e-40, -1.1254e-40,  1.9111e-41,  2.2774e-41, -1.0781e-40,
          4.6663e-40,  1.9149e-40, -8.8530e-41],
        [-5.1151e-17, -3.6522e-05,  4.4056e-37,  2.6964e-37, -1.9355e-06,
         -6.9483e-41, -3.5926e-41,  4.8288e-37, -5.8953e-42,  5.2027e-38,
          6.9370e-41,  6.5057e-41,  2.2735e-38],
        [-4.3567e-31, -4.2704e-04, -3.8406e-30,  3.7370e-30, -1.2069e-05,
          3.0258e-41, -5.2843e-41, -4.7967e-30, -2.9663e-31, -7.7387e-33,
          2.4121e-41, -1.1592e-40, -2.4645e-31],
        [-1.4264e-15, -2.8356e-04,  1.3576e-28,  3.4887e-29, -2.0630e-07,
          8.0119e-41, -5.5197e-42,  1.2111e-28,  5.9326e-30,  3.8508e-31,
         -6.6553e-41, -1.9503e-41,  6.7608e-30],
        [-3.0436e-42, -5.3058e-32,  1.5879e-40, -5.4605e-40,  6.0636e-41,
         -5.7403e-40,  1.1074e-40,  1.1032e-40,  9.2772e-41,  8.3531e-42,
          1.0157e-41,  2.9613e-40, -3.3410e-41],
        [-1.982

In [45]:
surgery['para']['layers.0.weight'][20:]
# surgery['para']['layers.0.bias']

tensor([[-7.5485e-03, -2.1980e-02, -2.7966e-05,  1.1425e-04, -2.0070e-02,
          4.0945e-09, -3.8096e-04, -1.1435e-03, -7.6633e-04, -2.3985e-07,
          1.8934e-07,  1.4813e-09, -1.0805e-03],
        [-9.6301e-01,  4.3007e-02, -1.3960e+00,  5.3394e-01,  1.8821e-01,
          3.9834e-01, -3.7659e-01, -2.6922e-01,  4.4095e-01, -5.6407e-01,
         -1.4727e-01, -1.7821e-01,  8.5678e-02],
        [-7.0375e-08, -9.4626e-03,  3.2083e-12, -4.3046e-14, -1.4371e-02,
         -2.1414e-07, -1.1120e-06,  4.0246e-10, -3.9200e-09,  2.3470e-09,
          5.8830e-13, -6.7900e-41, -7.5925e-07],
        [-9.1132e-01, -1.6379e-01,  3.6612e-01, -1.3511e+00,  3.5460e-01,
          7.5859e-01, -3.9762e-01, -2.5670e-01,  2.7064e-01, -6.2972e-01,
          5.6717e-02,  4.3588e-01, -5.4023e-01],
        [-3.2494e-01, -1.3964e-01, -1.2276e+00,  4.2912e-01,  3.8093e-01,
         -3.0529e-01,  3.5404e-02, -1.8604e-01, -4.7902e-01,  3.2775e-01,
          1.7182e-01,  5.7916e-03, -1.1977e-01],
        [ 0.000